In [35]:
import time

import matplotlib.pyplot as plt
import pandas as pd

from sklearn import cluster
from sklearn import metrics
from sklearn import preprocessing
from math import floor


#Activar para relizar pruebas con menos Datos
prueba = False



accidentes = pd.read_csv('accidentes_2013.csv')

if(prueba):
    accidentes = accidentes.sample(len(accidentes)//10)
    print(len(accidentes))
else:
    print(len(accidentes))


#seleccionar accidentes de tipo 'colisión de vehículos'
subset1 = accidentes[accidentes['TIPO_ACCIDENTE'].str.contains("Colisión de vehículos")]
subset1 = subset1[accidentes['TIPO_ACCIDENTE'].str.contains("(Alcance)") & (accidentes['TIPO_VIA'].str.contains("AUTOPISTA") | accidentes['TIPO_VIA'].str.contains("AUTOVÍA"))]
caso_estudio1 = ['HORA', 'DIASEMANA', 'TOT_VICTIMAS']
X1 = subset1[caso_estudio1]
print('Colisión accidentes: ',len(X1))
subset2 = accidentes[accidentes['TIPO_ACCIDENTE'].str.contains("Atropello")& accidentes['PROVINCIA'].str.contains("Córdoba")]

caso_estudio2 = ['MES', 'DIASEMANA','HORA', 'TOT_MUERTOS']
X2 = subset2[caso_estudio2]
print('Colisión accidentes: ',len(X2))
subset3 = accidentes[accidentes['TIPO_ACCIDENTE'].str.contains("Vuelco en la calzada")]
caso_estudio3 = ['TOT_HERIDOS_GRAVES', 'TOT_HERIDOS_LEVES', 'TOT_VEHICULOS_IMPLICADOS']
X3 = subset3[caso_estudio3]
print('Colisión accidentes: ',len(X3))

usadas = [caso_estudio1,caso_estudio2,caso_estudio3]
casos_de_estudio = [X1,X2,X3]

k_means = cluster.KMeans(init='k-means++',n_clusters=4, n_init=5)
mbkm = cluster.MiniBatchKMeans(n_clusters=4)
ms = cluster.MeanShift()
spectral = cluster.SpectralClustering(n_clusters=4)
affinity_propagation = cluster.AffinityPropagation()
dbscan = cluster.DBSCAN(eps=0.1, min_samples=5 )
birch = cluster.Birch(n_clusters=4,threshold=0.3)
ward = cluster.AgglomerativeClustering(n_clusters=4, linkage='ward')

clustering_algorithm = (
    ('k-means',k_means),
# #     ('MiniBatkMeans',mbkm),
    # ('MeanShift',ms)
    ('DBSCAN',dbscan),
    ('Birch',birch),
    ('SpectralClustering',spectral),
    ('Ward',ward)
)
for i,X in enumerate(casos_de_estudio):
    print("\nCaso de estudio "+str(i+1)+":\n")
    X_normal = preprocessing.normalize(X, norm='l2')

    for j,q in enumerate(clustering_algorithm):

        name,algorithm = q
        print('{:19s}'.format(name),end='')
        t = time.time()
        cluster_predict = algorithm.fit_predict(X_normal)
        # print('\nprediccion: ',cluster_predict)
        tiempo = time.time() - t
        k = len(set(cluster_predict))
        print(": k: {:3.0f}, ".format(k),end='')
        print("{:6.2f} segundos, ".format(tiempo),end='')

        if(k>1) and (name is not 'Ward'):
            metric_CH = metrics.calinski_harabaz_score(X_normal, cluster_predict)

            metric_SH = metrics.silhouette_score(X_normal, cluster_predict,metric = 'euclidean',sample_size=floor(0.1*len(X)),random_state=123456)
        else:
            metric_CH = 0
            metric_SH = 0
        print("CH Index: {:8.9f}, ".format(metric_CH),end='')
        print("SH: {:.5f}".format(metric_SH))

        #se convierte la asignación de clusters a DataFrame
        clusters = pd.DataFrame(cluster_predict,index=X.index,columns=['cluster'])
        # print('cluster: ',cluster)
        #y se añade como columna a X

        X_cluster = pd.concat([X, clusters], axis=1)
        #Filtro quitando los elementos (outliers) que caen en clusters muy pequeños en el jerárquico
        min_size = 3
        X_filtrado = X_cluster[X_cluster.groupby('cluster').cluster.transform(len) > min_size]
        k_filtrado = len(set(X_filtrado['cluster']))
        print('k_filtrado: ',k_filtrado)
        # print("De los {:.0f} clusters hay {:.0f} con más de {:.0f} elementos. Del total de {:.0f} elementos, se seleccionan {:.0f}".format(k['Ward'],k_filtrado,min_size,len(X),len(X_filtrado)))

        if(i==1):
            _X_filtrado_sin = X_filtrado.drop('cluster', 1)
            #Normalizo el conjunto filtrado
            _X_filtrado_normal = preprocessing.normalize(_X_filtrado_sin, norm='l2')
            #Ahora lo saco usando seaborn (que a su vez usa scipy) para incluir un heatmap
            import seaborn as sns
            _X_filtrado_normal_DF = pd.DataFrame(_X_filtrado_normal,index=_X_filtrado_sin.index,columns=usadas[i])
            dend_plot = sns.clustermap(_X_filtrado_normal_DF, method='ward', col_cluster=False, figsize=(20,10), cmap="YlGnBu")
            dend_plot.savefig('imagenes/dendograma.png')

        print("---------- Preparando el scatter matrix...")
        import seaborn as sns
        sns.set()
        variables = list(X_filtrado)
        variables.remove('cluster')
        sns_plot = sns.pairplot(X_filtrado, vars=variables, hue="cluster", palette='Paired', plot_kws={"s": 25}, diag_kind="hist") #en hue indicamos que la columna 'cluster' define los colores
        sns_plot.fig.subplots_adjust(wspace=.03, hspace=.03);
        sns_plot.savefig('imagenes/Caso_de_estudio_'+str(i)+'/'+clustering_algorithm[j][0]+".png")

89519


/home/ismael/anaconda3/envs/IN_/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/home/ismael/anaconda3/envs/IN_/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Colisión accidentes:  3222
Colisión accidentes:  169
Colisión accidentes:  3334

Caso de estudio 1:

k-means            : k:   4,   0.21 segundos, CH Index: 3780.191121307, SH: 0.41152
k_filtrado:  4
---------- Preparando el scatter matrix...


/home/ismael/anaconda3/envs/IN_/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


DBSCAN             : k:   3,   0.28 segundos, CH Index: 161.471105833, SH: 0.67063
k_filtrado:  3
---------- Preparando el scatter matrix...


...

---------- Preparando el scatter matrix...
SpectralClustering : k:   4,   0.14 segundos, CH Index: 161.891331377, SH: 0.23539
k_filtrado:  4
---------- Preparando el scatter matrix...
Ward               : k:   4,   0.00 segundos, CH Index: 0.000000000, SH: 0.00000
k_filtrado:  4
---------- Preparando el scatter matrix...

Caso de estudio 3:

k-means            : k:   4,   0.04 segundos, CH Index: 17961.363341743, SH: 0.94660
k_filtrado:  4
---------- Preparando el scatter matrix...
DBSCAN             : k:  12,   0.34 segundos, CH Index: 28321.997616825, SH: 0.97464
k_filtrado:  12
---------- Preparando el scatter matrix...


/home/ismael/anaconda3/envs/IN_/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Birch              

/home/ismael/anaconda3/envs/IN_/lib/python3.6/site-packages/sklearn/cluster/birch.py:629: UserWarning: Number of subclusters found (2) by Birch is less than (4). Decrease the threshold.
  % (len(centroids), self.n_clusters))


: k:   2,   0.24 segundos, CH Index: 9700.192260488, SH: 0.83179
k_filtrado:  2
---------- Preparando el scatter matrix...
SpectralClustering : k:   4,  41.81 segundos, CH Index: 10211.398881028, SH: 0.91628
k_filtrado:  4
---------- Preparando el scatter matrix...
Ward               : k:   4,   0.61 segundos, CH Index: 0.000000000, SH: 0.00000
k_filtrado:  4
---------- Preparando el scatter matrix...
